## IMPORTS

In [11]:
# top level imports
from pathlib import Path
import os, sys
import itertools

import parse

import napari

from skimage.measure import (regionprops, _regionprops)

from napari.utils.notebook_display import nbscreenshot

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files)

from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats import (_my_props_to_dict, _assert_uint16_labels, get_region_metrics_3D)
from infer_subc.utils.stats_helpers import *

from infer_subc.organelles import * 

from infer_subc.constants import (TEST_IMG_N,
                                    NUC_CH ,
                                    LYSO_CH ,
                                    MITO_CH ,
                                    GOLGI_CH ,
                                    PEROX_CH ,
                                    ER_CH ,
                                    LD_CH ,
                                    RESIDUAL_CH ) 

import time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get and load image for quantification
Specifically, this will include the raw image and the outputs from segmentation.

In [2]:
test_img_n = TEST_IMG_N

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"

raw_data_path = data_root_path / "raw"
im_type = ".czi"

raw_file_list = list_image_files(raw_data_path,im_type)
raw_img_name = raw_file_list[test_img_n]

# adding an additional list of image paths for the matching segmentation files
seg_data_path = data_root_path / "out"
seg_file_list = list_image_files(seg_data_path, "tiff")

# changing output directory for this notebook to a new folder called "quant"
out_data_path = data_root_path / "quant"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [3]:
# raw image
raw_img_data, raw_meta_dict = read_czi_image(raw_img_name)

channel_names = raw_meta_dict['name']
img = raw_meta_dict['metadata']['aicsimage']
scale = raw_meta_dict['scale']
channel_axis = raw_meta_dict['channel_axis']

In [4]:
## For each import, change the string to match the suffix on the segmentation files (i.e., the stuff following the "-")

# masks
nuc_seg = import_inferred_organelle("20230426_test_nuc", raw_meta_dict, seg_data_path)
cell_seg = import_inferred_organelle("20230426_test_cell", raw_meta_dict, seg_data_path)
cyto_seg = import_inferred_organelle("20230426_test_cyto", raw_meta_dict, seg_data_path)
# mask_seg = import_inferred_organelle("masks", raw_meta_dict, seg_data_path)

#organelles
lyso_seg = import_inferred_organelle("20230426_test_lyso", raw_meta_dict, seg_data_path)
mito_seg = import_inferred_organelle("20230426_test_mito", raw_meta_dict, seg_data_path)
golgi_seg = import_inferred_organelle("20230426_test_golgi", raw_meta_dict, seg_data_path)
perox_seg = import_inferred_organelle("20230426_test_perox", raw_meta_dict, seg_data_path)
ER_seg = import_inferred_organelle("20230426_test_ER", raw_meta_dict, seg_data_path)
LD_seg = import_inferred_organelle("20230426_test_LD", raw_meta_dict, seg_data_path)


loaded  inferred 3D `20230426_test_nuc`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_cell`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_cyto`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_lyso`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_mito`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_golgi`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_perox`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_ER`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_LD`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 


In [ ]:
# NEED TO DEPRICATE OR UPDATE THIS SO THAT THESE FUNCTIONS DO NOT CREATE A SEGMENTATION BASED ON HARD CODED STUFF

# ###################
# # CELLMASK, NUCLEI, CYTOPLASM, NUCLEUS
# ###################
# nuclei_obj =  get_nuclei(img_data,meta_dict, out_data_path)
# cellmask_obj = get_cellmask(img_data, nuclei_obj, meta_dict, out_data_path)
# cyto_mask = get_cytoplasm(nuclei_obj , cellmask_obj , meta_dict, out_data_path)

# putting it all together

`make_organelle_metrics_tables` prototype

### Testing how to make pairs of organelles with no redundancy

In [ ]:
test_lyso_metrics, rp = get_org_metrics_3D(lyso_seg, raw_img_data[LYSO_CH], cell_seg, scale)
test_lyso_metrics

In [6]:
# names of organelles we have
organelle_names = ["lyso", "mito","golgi","perox","ER","LD"]

# get the intensities
organelle_channels = [LYSO_CH,MITO_CH,GOLGI_CH,PEROX_CH,ER_CH,LD_CH]

# create intensities from raw as list
intensities = [raw_img_data[ch] for ch in organelle_channels]

# load organelles as list
organelles = [lyso_seg, mito_seg, golgi_seg, perox_seg, ER_seg, LD_seg]

In [ ]:
# merged_tabs = []
# for pair in test_contact_combos:

a_name = 'lyso' #pair[0]
b_name = 'mito' #pair[1]

i = organelle_names.index(a_name)
j = organelle_names.index(b_name)

a = _assert_uint16_labels(organelles[i]) # org_obj
b = _assert_uint16_labels(organelles[j])

ov = []
b_labs = []
labs = []
# for idx, lab in enumerate(a_stats_tab["label"]):  # a_stats_tab is the data associated to the a organelle collected earlier
for idx, lab in enumerate(test_lyso_metrics["label"]):
    xyz = tuple(rp[idx].coords.T) # change to xyz = tuple(rps[i].coords.T)
    cmp_org = b[xyz]
    
    # total number of overlapping pixels
    overlap = sum(cmp_org > 0)
    # overlap?
    labs_b = cmp_org[cmp_org > 0]
    b_js = np.unique(labs_b).tolist()

    # if overlap > 0:
    labs.append(id) # labs.append(lab)
    ov.append(overlap)
    b_labs.append(b_js)

# add organelle B columns to A_stats_tab
test_lyso_metrics[f"{b_name}_overlap"] = ov
test_lyso_metrics[f"{b_name}_labels"] = b_labs  # might want to make this easier for parsing later

#####  2  ###########
# get cross_stats

cross_tab = get_contact_metrics_3D(a, a_name, b, b_name, cell_seg) 
# shell_cross_tab = get_contact_metrics_3D(a, b, organelle_mask, use_shell_a=True)
            
# cross_tab["organelle_b"]=nmi
# shell_cross_tab["organelle_b"]=nmi
#  Merge cross_tabs and shell_cross_tabs 
# merged_tab = pd.merge(cross_tab,shell_cross_tab, on="label_")
# merged_tab = pd.concat([cross_tab,shell_cross_tab])
# cross_tab.insert(loc=0,column='organelle_b',value=b_name)

cross_tab

In [5]:
test_contact_combos = list(itertools.combinations(organelle_names, 2))
test_contact_combos

NameError: name 'organelle_names' is not defined

In [ ]:
test_contact_combos = list(itertools.combinations(organelle_names, 2))

contact_tabs = []
for pair in test_contact_combos:
    a_name = pair[0]
    b_name = pair[1]

    i = organelle_names.index(a_name)
    j = organelle_names.index(b_name)

    a = _assert_uint16_labels(organelles[i]) # org_obj
    b = _assert_uint16_labels(organelles[j])

    org_a_tab = org_tabs[i]
    org_a_rp = rps[i]

    ov = []
    b_labs = []
    labs = []
    # for idx, lab in enumerate(a_stats_tab["label"]):  # a_stats_tab is the data associated to the a organelle collected earlier
    for idx, lab in enumerate(org_a_tab["label"]):
        xyz = tuple(org_a_rp[idx].coords.T) # change to xyz = tuple(rps[i].coords.T)
        cmp_org = b[xyz]
        
        # total number of overlapping pixels
        overlap = sum(cmp_org > 0)
        # overlap?
        labs_b = cmp_org[cmp_org > 0]
        b_js = np.unique(labs_b).tolist()

        # if overlap > 0:
        labs.append(id) # labs.append(lab)
        ov.append(overlap)
        b_labs.append(b_js)

    cname = organelle_to_colname[b_name]


    cross_tab = get_contact_metrics_3D(a, a_name, b, b_name, cell_seg) 


    contact_tabs.append(cross_tab)

### Final function

In [23]:
import time

In [26]:
organelle_to_colname = {"nuc":"NU", "lyso": "LY", "mito":"MT", "golgi":"GL", "perox":"PR", "ER":"ER", "LD":"LD", "cell":"CM", "cyto":"CY", "nucleus": "N1","nuclei":"NU",}

def _make_all_metrics_tables(
    organelle_names: List[str],
    organelles: List[np.ndarray],
    intensities: List[np.ndarray],
    region_names: List[str],
    regions: List[np.ndarray],
    dist_centering_obj:np.ndarray, 
    mask:np.ndarray,
    source_file: str,
    scale: Union[tuple,None] = None,
    n_XY_bins: Union[int,None] = None,
    n_zernike: Union[int,None] = None,
    include_contact_dist: bool = True
) -> int:
    """
    get summary and all cross stats between organelles `a` and `b`
    calls `get_summary_stats_3D`
    order of intensity measurements in the regions dataframe is the same as the order of organelle names provided in the organelle_names list
    """
    start = time.time()
    count = 0
    ####################
    # collect individual organelle measurements, 
    # the organelle contacts associated the that organelle, 
    # and total organelle distribution measurements
    # for each organelle
    ####################
    org_tabs = []
    rps = []
    dist_tabs = []
    XY_bins = []

    # zern_bins = []
    for j, target in enumerate(organelle_names):
        org_img = intensities[j]
        if target == 'ER':
            org_obj = (organelles[j] > 0).astype(np.uint16)
        else:
            org_obj = organelles[j]

        ####################
        # measure individual organelle and cell region metrics
        ####################
        org_metrics, rp = get_org_metrics_3D(segmentation_img=org_obj, 
                                             intensity_img=org_img, 
                                             mask=mask,
                                             scale=scale)
        org_metrics.insert(loc=0,column='organelle',value=target)
        ### org_metrics.insert(loc=0,column='cell',value=1) 
        # ^^^ saving this thought for later when someone might have more than one cell per image.
        # Not sure how they analysis process would fit in our pipelines as they exist now. 
        # Maybe here, iterating though the index of the masks above all of this and using that index as the cell number?


        ####################
        # collect organelles in contact with this organelle
        ####################
        for i, nmi in enumerate(organelle_names):
            if i != j:
                if target == 'ER':
                    b = (organelles[i] > 0).astype(np.uint16)
                else:
                    b = organelles[i]
            
                ov = []
                b_labs = []
                labs = []
                for idx, lab in enumerate(org_metrics["label"]):
                    xyz = tuple(rp[idx].coords.T)
                    cmp_org = b[xyz]
                    
                    # total area (in voxels or real world units) where these two orgs overlap within the cell
                    if scale != None:
                        overlap = sum(cmp_org > 0)*scale[0]*scale[1]*scale[2]
                    else:
                        # total number of overlapping pixels
                        overlap = sum(cmp_org > 0)
                        # overlap?
                    
                    # which b organelles are involved in that overlap
                    labs_b = cmp_org[cmp_org > 0]
                    b_js = np.unique(labs_b).tolist()

                    # if overlap > 0:
                    labs.append(lab) # labs.append(lab)
                    ov.append(overlap)
                    b_labs.append(b_js)
                org_metrics[f"{nmi}_overlap"] = ov
                org_metrics[f"{nmi}_labels"] = b_labs 

        ####################
        # measure organelle distribution 
        ####################
        XY_org_distribution ,zernike_org_distribution, XY_bin_masks = get_XY_dist_metrics(cellmask_obj=mask, # TODO: add output for zernike masks
                                                                 nuclei_obj=dist_centering_obj,
                                                                 organelle_obj=org_obj,
                                                                 organelle_name=target,
                                                                 num_bins=n_XY_bins,
                                                                 zernike_degrees=n_zernike)

        Z_org_distribution = get_Z_dist_metrics(cellmask_obj=mask, 
                                            organelle_obj=org_obj,
                                            organelle_name=target,
                                            nuclei_obj=dist_centering_obj)
      
        org_distribution_metrics = pd.merge(XY_org_distribution, zernike_org_distribution,on=["organelle"])
        org_distribution_metrics = pd.merge(org_distribution_metrics, Z_org_distribution,on=["organelle"])

        org_tabs.append(org_metrics)
        rps.append(rp)
        dist_tabs.append(org_distribution_metrics)
        XY_bins.append(XY_bin_masks)
        # zern_bins.append(zernike_masks)
    
    ####################
    # measure individual organelle and cell region metrics
    ####################
    raw_image = np.stack(intensities)
    region_metrics = get_region_metrics_3D(region_segs=regions, 
                                        region_names=region_names,
                                        intensity_img=raw_image,
                                        mask=mask)

    ####################
    # collect non-redundant contact metrics
    #  TODO: figure out how to add the shell measurements as new columns, not rows    
    ####################
    contact_combos = list(itertools.combinations(organelle_names, 2))
    contact_tabs = []
    for pair in contact_combos:
        a_name = pair[0]
        b_name = pair[1]

        i = organelle_names.index(a_name)
        j = organelle_names.index(b_name)

        a = organelles[i] # org_obj
        b = organelles[j]

        if include_contact_dist == True:
            contact_tab, contact_XY_dist, contact_Zern_dist, contact_Z_dist = get_contact_metrics_3D(a, a_name, b, b_name, mask, 
                                                                                                     False, True, dist_centering_obj)
            
            contact_dist_metrics = pd.merge(contact_XY_dist, contact_Zern_dist,on=["organelle"])
            contact_dist_metrics = pd.merge(contact_dist_metrics, contact_Z_dist,on=["organelle"])
        else:
            contact_tab = get_contact_metrics_3D(a, a_name, b, b_name, mask, False)
            
        contact_tabs.append(contact_tab)
        dist_tabs.append(contact_dist_metrics)

    ####################
    # combine all tabs into one table per type:
    ####################
    final_org_tab = pd.concat(org_tabs, ignore_index=True)
    final_org_tab.insert(loc=0,column='image_name',value=source_file.stem )

    final_contact_tab = pd.concat(contact_tabs, ignore_index=True)
    final_contact_tab.insert(loc=0,column='image_name',value=source_file.stem )

    combined_dist_tab = pd.concat(dist_tabs, ignore_index=True)
    combined_dist_tab.insert(loc=0,column='image_name',value=source_file.stem )

    region_metrics.insert(loc=0,column='image_name',value=source_file.stem )

    end = time.time()
    print(f"It took {(end-start)/60} minutes to quantify one image.")
    return final_org_tab, final_contact_tab, combined_dist_tab, region_metrics

In [27]:
# TODO: things to fix - 
# figure out what is causing the convex hull error - I'm guessing major axis, but not sure
regions = [cell_seg, cyto_seg, nuc_seg]
region_names = ['cell', 'cyto', 'nuc']

test_final_org_tab, test_final_contact_tab, test_combined_dist_tab, test_final_regions_tab = _make_all_metrics_tables(organelle_names=organelle_names,
                                                                                                    organelles=organelles,
                                                                                                    intensities=intensities,
                                                                                                    regions=regions,
                                                                                                    region_names=region_names,
                                                                                                    dist_centering_obj=nuc_seg,
                                                                                                    mask=cell_seg,
                                                                                                    scale=scale,
                                                                                                    source_file= raw_img_name,
                                                                                                    n_XY_bins=5,
                                                                                                    n_zernike=9,
                                                                                                    include_contact_dist=True)

c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1891697903  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  6  Error-roundoff 8.1e-15  _one-merge 5.6e-14
  _near-inside 2.8e-13  Visible-distance 1.6e-14  U-max-coplanar 1.6e-14
  Width-outside 3.2e-14  _wide-facet 9.7e-14  _maxoutside 6.4e-14

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While ex

It took 2.70772971312205 minutes to quantify one image.


In [28]:
test_final_regions_tab

,image_name,region,label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length,min_intensity-0,min_intensity-1,min_intensity-2,min_intensity-3,min_intensity-4,min_intensity-5,max_intensity-0,max_intensity-1,max_intensity-2,max_intensity-3,max_intensity-4,max_intensity-5,mean_intensity-0,mean_intensity-1,mean_intensity-2,mean_intensity-3,mean_intensity-4,mean_intensity-5,standard_deviation_intensity-0,standard_deviation_intensity-1,standard_deviation_intensity-2,standard_deviation_intensity-3,standard_deviation_intensity-4,standard_deviation_intensity-5
0,a24hrs-Ctrl_14_Unmixing,cell,1,7.520524,373.314989,299.788285,0,0,0,16,659,592,1545233.0,285939.000000,143.438296,0.247552,-13,2467305.0,0.626284,682.673754,0.0,0.0,0.0,0.0,0.0,0.0,25817.0,43784.0,65535.0,25484.0,45278.0,43756.0,438.778382,1103.154897,968.052221,686.640820,2262.953316,615.328306,1176.228311,3162.565822,3291.795243,1287.589652,2878.302425,1163.771714
0,a24hrs-Ctrl_14_Unmixing,cyto,1,7.813587,366.605336,283.553501,0,0,0,16,659,592,1186798.0,348005.531250,131.358867,0.190130,-13,2467305.0,0.481010,750.817626,0.0,0.0,0.0,0.0,0.0,0.0,25817.0,43784.0,65535.0,25484.0,45278.0,43756.0,465.353326,1394.939359,1197.639393,771.723197,2617.119110,705.949372,1296.688966,3551.936736,3718.003627,1396.749632,3100.415351,1298.872167
0,a24hrs-Ctrl_14_Unmixing,nuc,5,6.756134,395.660493,352.441450,0,284,242,16,504,463,417950.0,64248.140625,92.763096,0.537266,1,457538.0,0.913476,264.582839,0.0,0.0,0.0,0.0,0.0,0.0,20433.0,15222.0,15249.0,15353.0,19653.0,7637.0,367.603307,159.087391,236.961146,473.744757,1447.220691,357.386144,652.078905,453.954104,496.541180,887.180795,1842.257956,442.751818


In [29]:
# TODO: fix these things -
# figure out where the NaN values are coming from in the org_overlap with ER 
#       this is happening because of the non-redundancy method - the combos that aren't included in the non-redundant pairs doesn't have data in the chart
pd.set_option('display.max_columns', None)

test_final_org_tab

,image_name,organelle,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity,mito_overlap,mito_labels,golgi_overlap,golgi_labels,perox_overlap,perox_labels,ER_overlap,ER_labels,LD_overlap,LD_labels,lyso_overlap,lyso_labels
0,a24hrs-Ctrl_14_Unmixing,lyso,1,"(0.3891, 0.0799, 0.0799)",0.000000,21.855788,15.930764,0,271,197,1,278,202,39.170113,0.054612,0.470721,0.628571,1,0.000000,inf,0.636832,0.0,6740.0,2750.772727,1982.010364,0.000000,[],0.000000,[],0.000000,[],0.000000,[],0.00000,[],NaN,NaN
1,a24hrs-Ctrl_14_Unmixing,lyso,2,"(0.3891, 0.0799, 0.0799)",0.000000,22.349087,19.423787,0,278,241,1,283,246,30.109638,0.039718,0.423314,0.640000,1,0.000000,inf,0.592344,174.0,4247.0,2239.250000,1050.838980,0.000000,[],0.000000,[],0.000000,[],0.000000,[],0.00000,[],NaN,NaN
2,a24hrs-Ctrl_14_Unmixing,lyso,3,"(0.3891, 0.0799, 0.0799)",0.000000,22.670667,16.512820,0,279,203,1,289,210,56.715870,0.076954,0.527728,0.442857,1,0.000000,inf,0.792587,315.0,4582.0,2201.580645,1087.198583,0.076954,[2],0.000000,[],0.000000,[],0.000000,[],0.00000,[],NaN,NaN
3,a24hrs-Ctrl_14_Unmixing,lyso,4,"(0.3891, 0.0799, 0.0799)",0.190035,28.081387,22.904589,0,348,283,2,357,290,80.230835,0.106742,0.588544,0.341270,1,0.151425,0.704918,0.924011,0.0,6208.0,2170.000000,1508.550745,0.022341,[2],0.000000,[],0.000000,[],0.000000,[],0.00000,[],NaN,NaN
4,a24hrs-Ctrl_14_Unmixing,lyso,5,"(0.3891, 0.0799, 0.0799)",0.440619,28.385563,24.257490,0,352,299,3,360,310,162.089188,0.337603,0.863911,0.515152,1,0.402145,0.839506,1.333907,0.0,8111.0,2645.058824,1895.408771,0.000000,[],0.000000,[],0.000000,[],0.000000,[],0.00000,[],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,a24hrs-Ctrl_14_Unmixing,perox,42,"(0.3891, 0.0799, 0.0799)",5.058540,36.221794,35.702628,13,452,446,14,456,449,40.826965,0.024824,0.361929,0.833333,1,0.000000,inf,0.391290,3727.0,9813.0,7179.700000,2008.598370,0.000000,[],0.000000,[],NaN,NaN,0.000000,[],0.00000,[],0.000000,[]
143,a24hrs-Ctrl_14_Unmixing,perox,43,"(0.3891, 0.0799, 0.0799)",5.058540,37.891112,34.041298,13,474,425,14,476,428,24.706017,0.012412,0.287263,0.833333,1,0.000000,inf,0.276684,5396.0,14727.0,10237.000000,4098.921663,0.000000,[],0.000000,[],NaN,NaN,0.000000,[],0.00000,[],0.000000,[]
144,a24hrs-Ctrl_14_Unmixing,perox,44,"(0.3891, 0.0799, 0.0799)",5.447659,24.001431,21.685153,14,300,271,15,302,273,20.241913,0.009929,0.266671,1.000000,1,0.000000,inf,0.178598,7855.0,14096.0,12243.000000,2576.774631,0.002482,[2],0.000000,[],NaN,NaN,0.004965,[3],0.00000,[],0.000000,[]
145,a24hrs-Ctrl_14_Unmixing,ER,1,"(0.3891, 0.0799, 0.0799)",2.698696,28.102147,22.608693,0,0,41,16,628,562,126364.679688,229.631999,7.597626,0.017670,-11,3459.438763,0.066378,47.058328,0.0,45278.0,8918.054516,4392.976511,24.384359,[1],3.214674,[1],0.290438,[1],NaN,NaN,0.78443,[1],3.733490,[1]


In [30]:
test_final_contact_tab

,image_name,label,X_label,org_A,org_A_label,org_B,org_B_label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length
0,a24hrs-Ctrl_14_Unmixing,1,3_2,lyso,3,mito,2,0.000000,283.838710,206.741935,0,279,203,1,289,210,31.0,56.715870,3.897514,0.442857,1,0.0,inf,9.923254
1,a24hrs-Ctrl_14_Unmixing,2,4_2,lyso,4,mito,2,0.888889,350.777778,287.111111,0,349,286,2,354,289,9.0,39.173515,2.580762,0.300000,1,12.0,0.750000,5.314464
2,a24hrs-Ctrl_14_Unmixing,3,11_2,lyso,11,mito,2,1.700000,285.566667,198.766667,1,283,195,3,289,203,30.0,121.070824,3.855146,0.312500,0,58.0,0.517241,10.382515
3,a24hrs-Ctrl_14_Unmixing,4,14_2,lyso,14,mito,2,2.400000,329.488889,255.211111,1,325,251,4,334,260,90.0,186.289124,5.560083,0.370370,1,117.0,0.769231,10.087265
4,a24hrs-Ctrl_14_Unmixing,5,26_2,lyso,26,mito,2,6.049296,293.563380,210.200704,2,281,201,9,305,225,284.0,739.604797,8.155299,0.070437,1,1135.0,0.250220,34.962384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,a24hrs-Ctrl_14_Unmixing,19,39_3,perox,39,ER,3,10.000000,321.000000,261.000000,10,321,261,11,322,262,1.0,6.928203,1.240701,1.000000,1,0.0,inf,0.000000
549,a24hrs-Ctrl_14_Unmixing,20,41_3,perox,41,ER,3,12.000000,482.666667,432.666667,12,482,432,13,484,434,3.0,17.049160,1.789400,0.750000,1,0.0,inf,2.581989
550,a24hrs-Ctrl_14_Unmixing,21,44_3,perox,44,ER,3,14.000000,301.000000,271.500000,14,301,271,15,302,273,2.0,12.585057,1.563185,1.000000,1,0.0,inf,2.236068
551,a24hrs-Ctrl_14_Unmixing,1,3_1,ER,3,LD,1,9.087379,291.928803,201.627832,7,284,196,12,300,211,309.0,453.044556,8.387901,0.257500,0,456.0,0.677632,18.260245


In [31]:
# TODO: fix the following things -
# FOR LATER - NOT IMPORTANT RIGHT NOW 
#       zernike_nuc_mag and .._phs aren't necessary
#       Consider how these numbers will be normalized downstream - ex- norm by cytoplasm or cell area?
#       Nuc could be useful in the future for people who use the cell or something else as the center object, but since we are assuming
#           that people who don't use the nucleus just don't have the nucleus, we are okay. Overall just needs more consideration
test_combined_dist_tab

,image_name,organelle,XY_n_bins,XY_bins,XY_cm_vox_cnt,XY_org_vox_cnt,XY_n_pix,XY_cm_cv,XY_org_cv,zernike_n,zernike_m,zernike_cm_mag,zernike_cm_phs,zernike_obj_mag,zernike_obj_phs,zernike_nuc_mag,zernike_nuc_phs,z_n,z_slice,z_cm_vox_cnt,z_org_vox_cnt,z_nuc_vox_cnt
0,a24hrs-Ctrl_14_Unmixing,lyso,5,"[1, 2, 3, 4, 5]","[668998, 148785, 195857, 247771, 283822]","[6304, 7883, 7900, 4186, 2800]","[43048.0, 11069.0, 15682.0, 24236.0, 67355.0]","[0.042582848531038074, 0.18347946069342405, 0....","[1.4160694203558115, 1.6995467154546646, 1.587...","[0, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 1, 0, 2, 1, 3, 0, 2, 4, 1, 3, 5, 0, 2, 4, ...","[1.0, 0.10297120932454752, 0.6963771048665276,...","[1.5707963267948966, 0.15033074821862005, -1.5...","[1.0, 0.15115709793366275, 0.8739707682287408,...","[1.5707963267948966, -1.917549451689745, -1.57...","[1.0, 0.20375001433482667, 0.862821721007766, ...","[1.5707963267948966, 0.7245025801269294, -1.57...",17,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[80332, 94600, 97091, 94546, 99604, 99762, 997...","[275, 516, 842, 1176, 1625, 2501, 2983, 3262, ...","[12202, 25237, 28976, 32115, 33644, 34610, 351..."
1,a24hrs-Ctrl_14_Unmixing,mito,5,"[1, 2, 3, 4, 5]","[668998, 148785, 195857, 247771, 283822]","[18925, 24604, 30828, 24291, 4482]","[43048.0, 11069.0, 15682.0, 24236.0, 67355.0]","[0.042582848531038074, 0.18347946069342405, 0....","[0.8298239511950027, 0.5681962343079784, 0.956...","[0, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 1, 0, 2, 1, 3, 0, 2, 4, 1, 3, 5, 0, 2, 4, ...","[1.0, 0.10297120932454752, 0.6963771048665276,...","[1.5707963267948966, 0.15033074821862005, -1.5...","[1.0, 0.10356494330286127, 0.8141053622534614,...","[1.5707963267948966, -1.770211972994114, -1.57...","[1.0, 0.20375001433482667, 0.862821721007766, ...","[1.5707963267948966, 0.7245025801269294, -1.57...",17,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[80332, 94600, 97091, 94546, 99604, 99762, 997...","[2359, 4010, 5367, 8065, 8932, 9326, 8842, 845...","[12202, 25237, 28976, 32115, 33644, 34610, 351..."
2,a24hrs-Ctrl_14_Unmixing,golgi,5,"[1, 2, 3, 4, 5]","[668998, 148785, 195857, 247771, 283822]","[2942, 9297, 7558, 278, 444]","[43048.0, 11069.0, 15682.0, 24236.0, 67355.0]","[0.042582848531038074, 0.18347946069342405, 0....","[2.2350068423799963, 1.8407716175167286, 2.042...","[0, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 1, 0, 2, 1, 3, 0, 2, 4, 1, 3, 5, 0, 2, 4, ...","[1.0, 0.10297120932454752, 0.6963771048665276,...","[1.5707963267948966, 0.15033074821862005, -1.5...","[1.0, 0.1691875675055311, 0.9207564086136578, ...","[1.5707963267948966, -2.053505459173055, -1.57...","[1.0, 0.20375001433482667, 0.862821721007766, ...","[1.5707963267948966, 0.7245025801269294, -1.57...",17,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[80332, 94600, 97091, 94546, 99604, 99762, 997...","[141, 221, 393, 1044, 1825, 2022, 1859, 1192, ...","[12202, 25237, 28976, 32115, 33644, 34610, 351..."
3,a24hrs-Ctrl_14_Unmixing,perox,5,"[1, 2, 3, 4, 5]","[668998, 148785, 195857, 247771, 283822]","[164, 61, 43, 30, 39]","[43048.0, 11069.0, 15682.0, 24236.0, 67355.0]","[0.042582848531038074, 0.18347946069342405, 0....","[0.9497479370466072, 1.6697137214356548, 1.731...","[0, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, ...","[0, 1, 0, 2, 1, 3, 0, 2, 4, 1, 3, 5, 0, 2, 4, ...","[1.0, 0.10297120932454752, 0.6963771048665276,...","[1.5707963267948966, 0.15033074821862005, -1.5...","[1.0, 0.08938382646971742, 0.8111314107098941,...","[1.5707963267948966, -0.6666491030802881, -1.5...","[1.0, 0.20375001433482667, 0.862821721007766, ...","[1.5707963267948966, 0.7245025801269294, -1.57...",17,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[80332, 94600, 97091, 94546, 99604, 99762, 997...","[0, 5, 15, 36, 33, 36, 76, 30, 0, 40, 43, 0, 4...","[12202, 25237, 28976, 32115, 33644, 34610, 351..."
4,a24hrs-Ctrl_14_Unmixing,ER,5,"[1, 2, 3, 4, 5]","[668998, 148785, 195857, 247771, 283822]","[33650, 14209, 20058, 18

In [34]:
from infer_subc.utils.stats_helpers import make_all_metrics_tables

official_org, official_contacts, official_dist, official_regions = make_all_metrics_tables(organelle_names=organelle_names,
                                                                                        organelles=organelles,
                                                                                        intensities=intensities,
                                                                                        regions=regions,
                                                                                        region_names=region_names,
                                                                                        dist_centering_obj=nuc_seg,
                                                                                        mask=cell_seg,
                                                                                        scale=scale,
                                                                                        source_file= raw_img_name,
                                                                                        n_XY_bins=5,
                                                                                        n_zernike=9,
                                                                                        include_contact_dist=True)

c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1896639161  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  6  Error-roundoff 8.1e-15  _one-merge 5.6e-14
  _near-inside 2.8e-13  Visible-distance 1.6e-14  U-max-coplanar 1.6e-14
  Width-outside 3.2e-14  _wide-facet 9.7e-14  _maxoutside 6.4e-14

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While ex

It took 2.75513242483139 minutes to quantify one image.


In [35]:
test_final_org_tab.equals(official_org), test_final_contact_tab.equals(official_contacts), test_combined_dist_tab.equals(official_dist), test_final_regions_tab.equals(official_regions)

(True, True, False, True)

In [36]:
test_combined_dist_tab.compare(official_dist)

zernike_obj_mag   
                                                 self   
8   [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...  \
15  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
19  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   

                                                       
                                                other  
8   [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...  
15  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...  
19  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...